<img style="float: right;" src="https://drive.google.com/uc?export=view&id=1qVA-sPHp5TY5D7enbU7mF4zZ8obu5wrC" width=150 height=150 />

# Water Balance
### CWatM is an open-source hydrological model developed at IIASA
<br><br>
## User inputs:
 - The folder path holding CWatM simulations.
 - Was MODFLOW activated?
 - The coordinates for the outlet of the basin

In [1]:
# The folder path holding CWatM simulations
#output_folder =  'C:/CWatM_output_Examples/CWatM_output_22Oct_01051995_01051996' #_Examples/CWatM_output_full2015_normal' #_Examples/Experiment1_withIrrigation_fullYear_31July2020' #_Bhima_17June20'
#output_folder = 'C:/CWatM_output_Examples/CWatM_output_oneYear_FallowAsStandard'
#output_folder = 'C:/GitHub/nira_repo/modules/hydro/hydro_outputs'
# If MODFLOW was activated for sub-surface simulation, set to True

#output_folder = 'C:/CWatM_output_Examples/CWatM_output_2005'
if not 'Modflow' in locals():
    Modflow = True
    
if not 'output_folder' in locals():
    output_folder = 'C:/CWatM_output'

#Modflow = False
#   The location coordinates of the final cell in the basin, the discharge point, 
latitude =  17.3875  #Bhima 17.3875    Nira 17.97, 75.12
longitude = 75.89583 #Bhima 75.89583  Nira 75.12


<img src="https://drive.google.com/uc?export=view&id=1teWUCVHsZTmfYqDL7jq8ZvfcTRUdE0t6" width=1200 height=1000 />

In [2]:
from netCDF4 import Dataset, num2date
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

import plotly
from plotly import graph_objs as go, offline as po, tools
from plotly.subplots import make_subplots
import plotly.express as px

#from PIL import Image
import datetime
import json
import pandas as pd

#from IPython.display import Image
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def commas(num):
    return '{:,.2f}'.format(num)

def geo_idx(dd, dd_array):
   """
     search for nearest decimal degree in an array of decimal degrees and return the index.
     np.argmin returns the indices of minium value along an axis.
     so subtract dd from all values in dd_array, take absolute value and find index of minium.
    """
   geo_idx = (np.abs(dd_array - dd)).argmin()
   return geo_idx

## Discharge at basin outlet

Here, we present the monthly discharge at the outlet of the Nira subbasin.

In [4]:
outlet = (latitude, longitude)

nc_simulated = Dataset(output_folder + '/discharge_daily.nc', 'r')

FLOWS_simulated = []

lats = nc_simulated.variables['lat'][:]
lons = nc_simulated.variables['lon'][:]

in_lat = outlet[0]
in_lon = outlet[1]

lat_idx = geo_idx(in_lat, lats)
lon_idx = geo_idx(in_lon, lons)

FLOWS_simulated.append(nc_simulated.variables['discharge'][:, lat_idx, lon_idx])


FileNotFoundError: [Errno 2] No such file or directory: b'C:/CWatM_output/discharge_daily.nc'

In [ ]:
Dates_simulation = num2date(nc_simulated.variables['time'][:], units=nc_simulated.variables['time'].units)
for t in range(len(Dates_simulation)):
    Dates_simulation[t] = np.datetime64(Dates_simulation[t])

fig_Discharge_outlet = go.Figure()

fig_Discharge_outlet.add_trace(go.Scatter(y=FLOWS_simulated[0],
                         x=Dates_simulation,
                mode='lines',
                name='Simulated'))


fig_Discharge_outlet.update_layout(title='Discharge at outlet',
                       xaxis_title = 'Days',
                       yaxis_title = 'Discharge (m3/s)')

#fig_Discharge_outlet.show()

In [ ]:
# ['CWatm variable name', 'Label', 'Unit', Input/Output/Storage']

# There are three different types of Units: 
# 1. Spatially districuted values in M
# 2. Spatially distributed values in M3
# 3. Summed total for the basin, where each cell holds the total sum for the basin

if Modflow == False:
    Vars = [
        ['storGroundwater',   'GW storage',       'M',      'Storage'],
        ['storGroundwater',   'GW storage',       'M',      'Storage_GW'],
        ['pumping', 'GW pumping', 'M', 'Output_GW']] #NEW
else:
    Vars = [
        ['gwstore',            'GW storage',       'summed', 'Storage'],
        ['gwstore',            'GW storage',       'summed', 'Storage_GW'],
        ['pumping_actual',     'GW pumping actual','summed', 'Output_GW']]
       
Vars.extend([
    # General Water Balance
    ['Rain',                  'Rain',                               'M',      'Input'],
    #['adjusted_channelFlow', 'Flow not available mid-day'           'M', 'Input'],
    #['Snow', 'Snow', 'M', "Input"]
    ['humanUse',          'Human use',                'M',     'Output'],
    ['channelStorage',        'Channel storage',                    'M3',     'Storage'],
    ['lakeResStorage',        'Water bodies storage',               'M3',     'Storage'],
    ['totalSto',              'Soil & Intercept storage',           'M',      'Storage'],
    #['gwstore',               'Groundwater storage',                'summed', 'Storage'],


    ['sum_actTransTotal',       'Transpiration',                'M',    'Output_ET'],
    ['actTransTotal_forest',    'Transpiration (Forest)',       'M',    'Output_Trans'], 
    ['actTransTotal_grasslands','Transpiration (Grasslands)',   'M',    'Output_Trans'], 
    ['actTransTotal_paddy',     'Transpiration (Paddy)',        'M',    'Output_Trans'], 
    ['actTransTotal_nonpaddy',  'Transpiration (non-Paddy)',    'M',    'Output_Trans'], 

    ['sum_actBareSoilEvap',     'Bare soil evaporation',        'M',    'Output_ET'],
    ['sum_interceptEvap',       'Intercept evaporation',        'M',    'Output_ET'],
    ['sum_openWaterEvap',       'Open water evaporation',       'M',    'Output_ET'],
    ['addtoevapotrans',         'Evap loss from withdrawals',   'M',    'Output_ET'],

    # Lakes and Reservoirs Water Balance
    ['lakeResInflowM',        'Waterbody inflow',               'M',    'Input_LR'],
    ['EvapWaterBodyM',        'Waterbody evaporation',          'M',    'Output_LR'],
    ['act_bigLakeResAbst',    'Waterbody abstraction, conveyed','M',    'Output_LR'],
    #act_LocalLakeAbstract
    #self.var.delivered_water
    ['leakage',               'Canal leakage',                  'M3',   'Output_LR'],
    ['lakeResOutflowM',       'Waterbody outflow',              'M',    'Output_LR'],
    ['lakeResStorage',        'Waterbody storage',              'M3',    'Storage_LR'],

    # Groundwater Water Balance
    
    #['leakage',          'Canal leakage',          'M3',     'Input_GW'],
    #['leakage',          'Canal leakage',          'M3',     'Input_gwRecharge_GW'], #NEW
    ['sum_gwRecharge',    'GW recharge',           'M',      'Input_GW'],
    #['pumping_actual',   'GW pumping actual',      'summed', 'Output_GW'],
    ['capillar',         'Capillary rise',         'M',      'Output_GW'],
    ['baseflow',         'Baseflow',               'M',      'Output_GW'],
    #['gwstore',          'GW storage',             'summed', 'Storage_GW'],
    ['riverbedExchangeM', 'Riverbed exchange', 'M', 'Input_gwRecharge_GW'],
    ['sum_prefFlow_GW', 'Preferential flow', 'M', 'Input_gwRecharge_GW'],
    ['sum_perc3toGW_GW', 'Soil percolation', 'M', 'Input_gwRecharge_GW'],
    ['leakageIntoGw', 'Leakage into GW', 'M', 'Input_gwRecharge_GW'],

    # Soil Water Balance
    ['Rain',                    'Rain',                   'M',  'Input_Soil'],
    ['riverbedExchangeM',       'Riverbed exchange',      'M',  'Input_Soil'],
    ['act_totalIrrConsumption', 'Irr consumption',        'M',   'Input_Soil'],
    ['capillar',                'Capillary rise',         'M',  'Input_Soil'],
    ['leakageIntoRunoff', 'Leakage into runoff', 'M', 'Input_Soil'],
    ['leakageIntoGw', 'Leakage into GW', 'M', 'Input_Soil'],
    #['EvapoChannel',          'Channel evaporation',      'M3', 'Output_Soil'],
    ['sum_actTransTotal',       'Transpiration',          'M',  'Output_Soil'],
    ['sum_actBareSoilEvap',     'Bare soil evaporation',  'M',  'Output_Soil'],
    ['sum_interceptEvap',       'Intercept evaporation',  'M',  'Output_Soil'],
    ['sum_openWaterEvap',       'Open water evaporation', 'M',  'Output_Soil'],
    ['sum_runoff',              'Runoff',                 'M', 'Output_Soil'],
    ['sum_gwRecharge',          'GW recharge',            'M',  'Output_Soil'],
    ['totalSto',                'Soil & Intercept storage','M',  'Storage_Soil'],
    
    ['act_irrWithdrawal', 'Withdrawal Irr', 'M', 'Input_Withdrawal'],
    ['act_nonIrrWithdrawal', 'Withdrawal nonIrr', 'M', 'Input_Withdrawal'],
    ['act_domWithdrawal',   'Domestic Withdrawal',          'M',  'Input_nonIrrWithdrawal'],
    ['act_indWithdrawal',   'Industrial Withdrawal',          'M',  'Input_nonIrrWithdrawal'],
    ['act_livWithdrawal',   'Livestock Withdrawal',          'M',  'Input_nonIrrWithdrawal'],

    ['act_nonIrrWithdrawalSW', 'Withdrawal SW nonIrr', 'M', 'Output_Withdrawal'],
    ['act_nonIrrWithdrawalGW', 'Withdrawal GW nonIrr', 'M', 'Output_Withdrawal'],
    ['act_irrWithdrawalSW',    'Withdrawal SW Irr', 'M', 'Output_Withdrawal'],
    ['act_irrWithdrawalGW', 'Withdrawal GW Irr', 'M', 'Output_Withdrawal'],
 

    # Consumption Water Balance
    ['act_nonIrrConsumption',   'nonIrr consumption',          'M',  'Output_Consumption'],
    ['act_domConsumption',   'Domesticconsumption',          'M',  'Output_nonIrrConsumption'],
    ['act_indConsumption',   'Industrial consumption',          'M',  'Output_nonIrrConsumption'],
    ['act_livConsumption',   'Livestock consumption',          'M',  'Output_nonIrrConsumption'],

    
    ['act_totalIrrConsumption', 'Irr consumption',             'M',  'Output_Consumption'],
    ['addtoevapotrans',         'Evap loss from withdrawals',  'M',  'Output_Consumption'],
    ['returnFlow',              'Return flow from withdrawals','M',  'Output_Consumption'],
    ['pumping',                 'Groundwater abstractions',                'M',  'Input_Consumption'], #GW requested of modflow
    ['act_SurfaceWaterAbstract','SW abstractions (applied)',   'M',  'Input_Consumption'],

    ['act_channelAbstract','Channel abstractions',   'M',  'Input_SW_Consumption'],
    ['act_bigLakeResAbst','Waterbody abstraction, conveyed',   'M',  'Input_SW_Consumption'],
    
    ['act_channelAbstract_Local','Channel (local) abstractions',   'M',  'Input_Channel_Consumption'],
    ['act_nonLocalchannelAbstract_M','Channel (non-local) abstractions',   'M',  'Input_Channel_Consumption'],
    ['act_LocalLakeAbstract','Waterbody (local) abstraction, conveyed',   'M',  'Input_Lake_Consumption'], 
    ['act_nonLocalLakeAbstract_M','Waterbody (non-local) abstraction, conveyed',   'M',  'Input_Lake_Consumption']])


In [ ]:
# Loading the data

import numpy.ma as ma

# WB = {Inputs:  [ ['Rain', [rain_value_day0, ..., rain_value_dayLast]] ],
#       Outputs: [ ['Evapotranspiration (soil)', [ET_value_day0, ..., ET_value_dayLast]] 
#                  ['Channel evaporation', [ChannelE_value_day0, ..., ChannelE_value_dayLast]], ... ]}

keys = [i[-1] for i in Vars]
WB = {key: [] for key in keys} # Creates an empty set relating to each key, overriding the previous to leave only one of each 
#print('keys: ', keys)
cellArea = Dataset(output_folder +'/'+ 'cellArea_daily.nc', 'r').variables['cellArea'][0,:,:]
cellAreaC = ma.compressed(cellArea)
#print('basin area', np.sum(cellAreaC))
for var in Vars:
    
    nc_filename = output_folder +'/'+ var[0] + '_daily.nc'
    temp = Dataset(nc_filename, 'r').variables[var[0]][:,:,:]
    Daily_basinSum = []
    print(var[0])
    for d in range(len(Dates_simulation)):
        
        if var[2] == 'M':
            daily_basinSum = np.sum(np.multiply(ma.compressed(temp[d,:,:]), cellAreaC))
        elif var[2] == 'M3':
            daily_basinSum = np.sum(ma.compressed(temp[d,:,:]))
        else:
            #All cells hold the same total-value for the basin
            daily_basinSum = ma.compressed(temp[d,:,:])[0]
            
        Daily_basinSum.append(daily_basinSum)
    
    WB[var[-1]].append([var[1], np.array(Daily_basinSum)])
    

In [ ]:
#When pumping is requested, the groundwater demand may exceed the available groundwater supply

# Pumping in M, Leakage in M3, and Actual in Summed
if Modflow == True:
    temp_Y_GWrequest = Dataset(output_folder +'/'+ 'pumping_daily.nc', 'r').variables['pumping'][:,:,:]
    temp_Y_GWactual = Dataset(output_folder +'/'+ 'pumping_actual_daily.nc', 'r').variables['pumping_actual'][:,:,:] #This is a single value
    temp_Y_GWleaked = Dataset(output_folder +'/'+ 'leakage_daily.nc', 'r').variables['leakage'][:,:,:]

    Y_GWrequest = []
    Y_GWleaked = []
    Y_GWactual = []


    for d in range(len(Dates_simulation)):

        Y_GWrequest.append(np.sum(np.multiply(ma.compressed(temp_Y_GWrequest[d,:,:]), cellAreaC)))
        Y_GWleaked.append(np.sum(ma.compressed(temp_Y_GWleaked[d,:,:])))
        Y_GWactual.append(np.array(ma.compressed(temp_Y_GWactual[d,:,:])[0]))

    Y_GWrequest = np.array(Y_GWrequest)[2:]
    Y_GWleaked = np.array(Y_GWleaked)[2:]
    Y_GWactual = np.array(Y_GWactual)[2:]


    Y_diffPumping = Y_GWrequest - Y_GWactual
    Y_diffPumping_array = Y_diffPumping[:] 
    #MS 18052020 
    # Adds abs for negative difference. Pumping request and actual are never perfect, and there are two reasons:
    # There may not be sufficient water
    # There is sometimes somewhat more pumped than requested. Assuming then there is perhaps sometimes less requested, but not only due to unsufficient supply. 
    # We are interested in the magnitude of the sum. 
    # Self-correction from cancelations are in our benefit
    Y_diffPumping = abs(np.sum(Y_diffPumping))
    
else:
    
    temp_Y_GWrequest = Dataset(output_folder +'/'+ 'pumping_daily.nc', 'r').variables['pumping'][:,:,:]
    
    Y_GWrequest = []
    for d in range(len(Dates_simulation)):
        Y_GWrequest.append(np.sum(np.multiply(ma.compressed(temp_Y_GWrequest[d,:,:]), cellAreaC)))
    
    Y_GWrequest = np.array(Y_GWrequest)[2:]
    Y_GWactual = Y_GWrequest.copy()
    Y_GWleaked = np.array([0]*len(Y_GWrequest))
    
    Y_diffPumping = 0
    
    

In [ ]:
X = Dates_simulation
X = X[2:]

fig = go.Figure()
fig.add_trace(go.Scatter(y= Y_GWrequest,
                         x = X,
                         mode='lines',
                        name='Groundwater abstractions'))
fig.add_trace(go.Scatter(y= Y_GWactual,
                         x = X,
                         mode='lines',
                        name='GW actual'))

fig.add_trace(go.Scatter(y= Y_GWleaked,
                         x = X,
                         mode='lines',
                        name='Canal leakage'))

fig.update_layout(title = 'Actual versus Requested pumping',xaxis_title = 'Days',yaxis_title = 'M3')

#fig.show()


# Overall Water Balance

In [ ]:
Figures = {}
X = Dates_simulation[2:]
Title = 'Overall Water Balance'

discharge_included = False

fig_Waterscape = go.Figure()
fig_Waterscape.update_layout(title = 'Water flows, daily: Waterscape<br><br> '+Title, xaxis_title = 'Days',yaxis_title = 'Flow (M3/day)', barmode='relative')

Total_input, Total_output, Total_change = [],[],[]
labels_out, parents_out, values_out, colors_out = [], [], [], []
labels_simple, parents_simple, values_simple, colors_simple = [], [], [], []
Input_daily, Output_daily, Change_daily = np.zeros(len(X)), np.zeros(len(X)), np.zeros(len(X))

Input_current = WB['Input']
Output_current = WB['Output']
Storage_current = WB['Storage']

VARS = [[Input_current, 'Inputs', Total_input, 1, '', Input_daily], 
        [Output_current, 'Outputs', Total_output, -1, '', Output_daily], 
        [Storage_current, 'Storage', Total_change, -1, ' Change', Change_daily] ]

Y_select = []
for VAR in VARS:
    for Var in VAR[0]:

        if VAR[1] == 'Outputs' and not discharge_included:
            Y = FLOWS_simulated[0] * 60*60*24
            Y =   Y[2:] #Y[1:-1] #Y[2:] # Experiment Y[1:-1] 
            
            Y_select.append([Y, 'River discharge at outlet'])
            
            VAR[5] += Y
            VAR[2].append(np.sum(Y))
            
            fig_Waterscape.add_trace(go.Bar(
                y = Y*VAR[3],
                x = X,
                name = 'River discharge at outlet'))

            labels_out.append('River outlet')
            parents_out.append('Water flows total')
            #parents_out.append(VAR[1])
            values_out.append(np.sum(Y))
            colors_out.append('river') #blue') ##636EFA') #negative')

            discharge_included = True

        # Put original Y into Select chart
        Y = Var[1]
        Y = Y[2:]
        Y_select.append([Y,Var[0]])
        if Var[0] == 'Evapotranspiration (soil)':
            total_ET = np.sum(Y)
        
        if VAR[1] == 'Storage':
            
            preY = np.append(Y[0], Var[1][:-1]) 
            Y = Y - preY[2:]
            """
            labels_out.append(Var[0]+VAR[4])
            parents_out.append(VAR[1]+VAR[4])
            values_out.append(abs(np.sum(Y)))
            colors_out.append('positive' if np.sum(Y)>0 else 'negative')
            """
            
            
        else:
            labels_out.append(Var[0]+VAR[4])
            #parents_out.append(VAR[1])
            parents_out.append('Water flows total')
            values_out.append(np.sum(Y))
            colors_out.append('positive' if VAR[1] == 'Inputs' else 'negative')
            
            labels_simple.append(Var[0]+VAR[4])
            parents_simple.append(VAR[1])
            values_simple.append(np.sum(Y))
            colors_simple.append('positive' if VAR[1] == 'Inputs' else 'negative')
        
            
        VAR[2].append(np.sum(Y)) # Sum of a single variable, over the simulation period
        VAR[5] += Y              # The daily total sum for all variables of a type: Inputs, Outputs, Change
        
    
        fig_Waterscape.add_trace(go.Bar(
            y = Y*VAR[3],
            x = X,
            name = Var[0]+VAR[4]))

        
    line = go.Scatter(x=X, y=Y_select[0][0])
    updatemenus = [{'buttons': [{'method': 'restyle','label': i[1],'args': [{'y': [i[0]]}]} for i in Y_select], 
                    'direction': 'down', 'pad':{"r": 10, "t": 10}, 'showactive': True, 'x':0.2, 'xanchor':'left', 'y':1.225, 'yanchor':'top'}]

    layout = go.Layout(
        updatemenus=updatemenus,
        title = 'Specific flows and storages',
        xaxis_title = 'Days',
        yaxis_title = 'Volume (M3) or Flow (M3/day)')

    figure = go.Figure(data=[line], layout=layout)
    figure.update_layout(template='plotly_dark')

total_input = sum(Total_input)
#MS 18052020 removed Y_diff
total_input_adjusted = total_input + Y_diffPumping
total_output = sum(Total_output)
total_change = sum(Total_change)

balance = total_input_adjusted - total_output - total_change
magnitude_change = np.sum([abs(i) for i in VARS[2][2]])
storageChangeActual = abs(np.sum([i for i in VARS[2][2]]))

percent_accurate_WB = 100*(balance) / (0.5*total_input_adjusted + 0.5*abs(total_change) + 0.5*total_output)



#################### 
# CIRCLES
#################### 
colors_Circle_main_discrete_map={'river': '#616BB2', 'positive':'#67C4EF','negative':'#EF5538', 'white':'#00CC96', 'storage':'#92A2AD', 'black':'black'}
values_Parents_map_Circle = {'Water flows total':total_input_adjusted+ total_output+storageChangeActual+abs(balance),
                             'Inputs':total_input_adjusted,   ##636EFA'
                             'Outputs':total_output+abs(balance) ,
                             'Storage Change':storageChangeActual,
                             '':storageChangeActual+abs(balance)+Y_diffPumping+sum(values_out) #total_input_adjusted+ total_output+magnitude_change+abs(balance)                            
                            }
####################   
# Circle, main
"""
labels_Circle_main =  ['Water flows total']+['Inputs',   'Outputs',  'Storage Change',     'Nature use',  'Adjusted pumping'] + labels_out 
parents_Circle_main =  ['']+['Water flows total',   'Water flows total',  'Water flows total',  'Outputs', 'Inputs']         + parents_out
values_Circle_main =    [total_input_adjusted+ total_output+storageChangeActual+abs(balance)]+[total_input_adjusted,  total_output+abs(balance),         storageChangeActual,    abs(balance),               Y_diffPumping    ] + values_out 

values_Parents_Circle_main = ["{:.1%}".format(values_Circle_main[i]/values_Parents_map_Circle[parents_Circle_main[i]]) for i in range(len(values_Circle_main))]
#colors_Circle_main =    ['white']+['positive','negative'] +  ['positive' if total_change>0 else 'negative']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
#colors_Circle_main =  ['white']+['positive','negative'] +  ['storage']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
colors_Circle_main =  ['black']+['positive','negative'] +  ['storage']+ ['white'] + ['positive'] + colors_out
"""
"""
labels_Circle_main =  ['Water flows total']+['Storage Change',     'Nature use',  'Adjusted pumping'] + labels_out 
parents_Circle_main =  ['']+['Water flows total',   'Water flows total',  'Water flows total']         + parents_out
#values_Circle_main =    [total_input_adjusted+ total_output+storageChangeActual+abs(balance)]+[ storageChangeActual,    abs(balance),               Y_diffPumping    ] + values_out 
values_Circle_main = [ storageChangeActual+abs(balance)+Y_diffPumping+sum(values_out)] + [ storageChangeActual,    abs(balance),               Y_diffPumping    ] + values_out 
values_Parents_Circle_main = ["{:.1%}".format(values_Circle_main[i]/values_Parents_map_Circle[parents_Circle_main[i]]) for i in range(len(values_Circle_main))]
#colors_Circle_main =    ['white']+['positive','negative'] +  ['positive' if total_change>0 else 'negative']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
#colors_Circle_main =  ['white']+['positive','negative'] +  ['storage']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
colors_Circle_main =  ['black']+ ['storage']+ ['white'] + ['positive'] + colors_out
"""
labels_Circle_main =  ['Storage Change',     'Nature use'] + labels_out 
parents_Circle_main =  ['Water flows total',   'Water flows total', ]         + parents_out
#values_Circle_main =    [total_input_adjusted+ total_output+storageChangeActual+abs(balance)]+[ storageChangeActual,    abs(balance),               Y_diffPumping    ] + values_out 
values_Circle_main = [ storageChangeActual,    abs(balance)  ] + values_out 
values_Parents_Circle_main = ["{:.1%}".format(values_Circle_main[i]/values_Parents_map_Circle[parents_Circle_main[i]]) for i in range(len(values_Circle_main))]
#colors_Circle_main =    ['white']+['positive','negative'] +  ['positive' if total_change>0 else 'negative']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
#colors_Circle_main =  ['white']+['positive','negative'] +  ['storage']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
colors_Circle_main =  ['storage']+ ['white'] + colors_out
labels  =labels_Circle_main
parents =parents_Circle_main
values  =values_Circle_main
colors  =colors_Circle_main
fractions = values_Parents_Circle_main
print(labels)
print(fractions)
print(colors)


d={'labels':labels, 'parents':parents, 'values':values, 'colors':colors, 'fractions':fractions}

df = pd.DataFrame(data=d)
"""
fig = px.sunburst(df, names='labels', parents='parents', values='values', color='colors', branchvalues='total', color_discrete_map=colors_Circle_main_discrete_map, hover_data=['fractions'])
#fig.update_layout(uniformtext=dict(minsize=15, mode='hide'))
fig.update_layout(height=700, title='Water Balance Circle,<br> whole<br><br> '+Title)
fig.update_layout(
    title={'y':0.75, 'x':0.05, 'xanchor': 'left', 'yanchor': 'top'}) #, 'font_size':26})

fig.update_layout(hoverlabel=dict(font_size=16), template='plotly_dark')
fig.show()      
"""
fig_pie = px.pie(df, names='labels', hole=.45, values='values',  color='colors', color_discrete_map=colors_Circle_main_discrete_map, hover_data=['fractions'])
fig_pie.update_traces(textinfo='label', marker=dict(line=dict(color='#000000', width=1.7))) #, pull=[0,0,0,0,0.2])
fig_pie.update_layout(hoverlabel=dict(font_size=16), template='plotly_dark')
fig_pie.show()
#{'river': '#616BB2', 'positive':'#67C4EF','negative':'#EF5538', 'white':'#00CC96', 'storage':'#92A2AD', 'black':'black'}

_labels=labels
_values = values
_colors=['#92A2AD', '#00CC96', '#67C4EF', '#616BB2', '#EF5538']

labels = [_labels[i] for i in [2,4,1,3,0]]
values = [_values[i] for i in [2,4,1,3,0]]
colors = [_colors[i] for i in [2,4,1,3,0]]

print(labels)
print(fractions)
print(colors)

#Security wheel
fig = go.Figure(
    data=[go.Pie(
        labels=labels, #df['labels'],
        values= values, #df['values'],
        # Second, make sure that Plotly won't reorder your data while plotting
        sort=False,
        rotation = -90,
    hole=.45)
    ])

#fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
#                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_traces(textinfo='label', marker=dict(colors=colors, line=dict(color='#000000', width=1.7))) #, pull=[0,0,0,0,0.2])
fig.update_layout(hoverlabel=dict(font_size=16), template='plotly_dark')
fig.show()

In [ ]:
#fig_Circle_complete.show()

## Specific water balances

## Waterbody

# Try all specific balances

In [ ]:
WB['Storage_Consumption']=[]
WB['Storage_Withdrawal']=[]
for section in [['Groundwater', WB['Input_GW'], WB['Output_GW'], WB['Storage_GW']], 
                ['Lakes and Reservoirs', WB['Input_LR'], WB['Output_LR'], WB['Storage_LR']],
                ['Soil', WB['Input_Soil'], WB['Output_Soil'], WB['Storage_Soil']],
                ['Withdrawal', WB['Input_Withdrawal'], WB['Output_Withdrawal'], WB['Storage_Withdrawal']],
                ['Consumption', WB['Input_Consumption'], WB['Output_Consumption'], WB['Storage_Consumption']]]:

    Title = section[0]+'-specifc'
    if section[0] == 'Consumption':
        Title = 'Consumption & Source'

    Input_current = section[1] #WB['Input_LR']
    Output_current = section[2]#WB['Output_LR']
    Storage_current = section[3]#WB['Storage_LR']

    X = Dates_simulation[2:]
    fig_Waterscape = go.Figure()
    fig_Waterscape.update_layout(title = 'Water flows, daily: Waterscape<br><br> '+Title, xaxis_title = 'Days',yaxis_title = 'Flow (M3/day)', barmode='relative')

    Total_input, Total_output, Total_change = [],[],[]
    labels_out, parents_out, values_out, colors_out = [], [], [], []
    labels_simple, parents_simple, values_simple, colors_simple = [], [], [], []
    Input_daily, Output_daily, Change_daily = np.zeros(len(X)), np.zeros(len(X)), np.zeros(len(X))



    VARS = [[Input_current, 'Inputs', Total_input, 1, '', Input_daily], 
            [Output_current, 'Outputs', Total_output, -1, '', Output_daily], 
            [Storage_current, 'Storage', Total_change, -1, ' Change', Change_daily] ]

    Y_select = []
    for VAR in VARS:
        for Var in VAR[0]:

            if Var[0]=='GW pumping actual':
               
                Y = Y_GWactual #+ Y_GWleaked
            else:
                # Put original Y into Select chart
                Y = Var[1]
                Y = Y[2:]

            Y_select.append([Y,Var[0]])

            if VAR[1] == 'Storage':

                preY = np.append(Y[0], Var[1][:-1]) 
                Y = Y - preY[2:]

                labels_out.append(Var[0]+VAR[4])
                parents_out.append(VAR[1]+VAR[4])
                values_out.append(abs(np.sum(Y)))
                colors_out.append('positive' if np.sum(Y)>0 else 'negative')


            else:
                labels_out.append(Var[0]+VAR[4])
                parents_out.append(VAR[1])
                values_out.append(np.sum(Y))
                colors_out.append('positive' if VAR[1] == 'Inputs' else 'negative')

                labels_simple.append(Var[0]+VAR[4])
                parents_simple.append(VAR[1])
                values_simple.append(np.sum(Y))
                colors_simple.append('positive' if VAR[1] == 'Inputs' else 'negative')


            VAR[2].append(np.sum(Y)) # Sum of a single variable, over the simulation period
            VAR[5] += Y              # The daily total sum for all variables of a type: Inputs, Outputs, Change


            fig_Waterscape.add_trace(go.Bar(
                y = Y*VAR[3],
                x = X,
                name = Var[0]+VAR[4]))

        if Y_select != []:
            line = go.Scatter(x=X, y=Y_select[0][0])
            updatemenus = [{'buttons': [{'method': 'restyle','label': i[1],'args': [{'y': [i[0]]}]} for i in Y_select], 
                            'direction': 'down', 'pad':{"r": 10, "t": 10}, 'showactive': True, 'x':0.2, 'xanchor':'left', 'y':1.225, 'yanchor':'top'}]

            layout = go.Layout(
                updatemenus=updatemenus,
                title = 'Specific flows and storages',
                xaxis_title = 'Days',
                yaxis_title = 'Volume (M3) or Flow (M3/day)')

            figure = go.Figure(data=[line], layout=layout)
            figure.update_layout(template='plotly_dark')

    total_input = sum(Total_input)
    total_input_adjusted = total_input #+ Y_diffPumping
    total_output = sum(Total_output)
    total_change = sum(Total_change)

    balance = total_input_adjusted - total_output - total_change
    magnitude_change = np.sum([abs(i) for i in VARS[2][2]])

    percent_accurate_WB = 100*(balance) / (0.5*total_input_adjusted + 0.5*abs(total_change) + 0.5*total_output)



    #################### 
    # CIRCLES
    #################### 
    colors_Circle_main_discrete_map={'positive':'#636EFA','negative':'#EF5538', 'white':'black', 'storage':'#00CC96'}
    values_Parents_map_Circle = {'Water flows total':total_input_adjusted+ total_output+magnitude_change+abs(balance),
                                 'Inputs':total_input_adjusted, 
                                 'Outputs':total_output,
                                 'Storage Change':magnitude_change,
                                 '':total_input_adjusted+ total_output+magnitude_change+abs(balance),
                                 'SW abstractions (applied)': np.sum(WB['Input_Consumption'][1][1][2:]),
                                 'Channel abstractions' : np.sum(WB['Input_SW_Consumption'][0][1][2:]),
                                 'Waterbody abstraction, conveyed' : np.sum(WB['Input_SW_Consumption'][1][1][2:]),
                                 'GW recharge' : np.sum(WB['Input_gwRecharge_GW'][0][1][2:])+np.sum(WB['Input_gwRecharge_GW'][1][1][2:])+np.sum(WB['Input_gwRecharge_GW'][2][1][2:])+np.sum(WB['Input_gwRecharge_GW'][3][1][2:]),
                                 'nonIrr consumption': np.sum(WB['Output_nonIrrConsumption'][0][1][2:]) + np.sum(WB['Output_nonIrrConsumption'][1][1][2:]) + np.sum(WB['Output_nonIrrConsumption'][2][1][2:]),
                                 'Withdrawal nonIrr': np.sum(WB['Input_nonIrrWithdrawal'][0][1][2:]) + np.sum(WB['Input_nonIrrWithdrawal'][1][1][2:]) + np.sum(WB['Input_nonIrrWithdrawal'][2][1][2:])
                                
                                }
    ####################   
    # Circle, main
    
    #    ['sum_prefFlow_GW', 'Preferential flow', 'M', 'Input_gwRecharge_GW'],
    # ['sum_perc3toGW_GW', 'Soil percolation', 'M', 'Input_gwRecharge_GW'],
    """
    if section[0] == 'Groundwater':
        labels_out.extend(['Riverbed exchange'])
        parents_out.extend(['GW recharge'])
        values_out.extend([np.sum(WB['Input_gwRecharge_GW'][0][1][2:])])
        colors_out.extend(['positive'])
   """
    if section[0] == 'Groundwater':
        labels_out.extend(['Riverbed exchange', 'Preferential flow', 'Soil percolation', 'Leakage into GW'])
        parents_out.extend(['GW recharge', 'GW recharge', 'GW recharge', 'GW recharge'])
        values_out.extend([np.sum(WB['Input_gwRecharge_GW'][0][1][2:]), np.sum(WB['Input_gwRecharge_GW'][1][1][2:]), np.sum(WB['Input_gwRecharge_GW'][2][1][2:]), np.sum(WB['Input_gwRecharge_GW'][3][1][2:])])
        colors_out.extend(['positive', 'positive', 'positive', 'positive'])
        
    if section[0] == 'Consumption':
        
        #act_LocalLakeAbstract, act_channelAbstract_Local
        #     ['act_channelAbstract_Local','Channel (local) abstractions',   'M',  'Input_Channel_Consumption'],
        #['act_LocalLakeAbstract','Waterbody (local) abstraction, conveyed',   'M',  'Input_Lake_Consumption']])
        #    ['act_channelAbstract_Local','Channel (local) abstractions',   'M',  'Input_Channel_Consumption'],
        #['act_nonLocalchannelAbstract_M','Channel (non-local) abstractions',   'M',  'Input_Channel_Consumption'],
        #['act_LocalLakeAbstract','Waterbody (local) abstraction, conveyed',   'M',  'Input_Lake_Consumption'], 
        #['act_nonLocalLakeAbstract_M','Waterbody (non-local) abstraction, conveyed',   'M',  'Input_Lake_Consumption']])
        
        # Experiment  ['act_nonIrrConsumption',   'nonIrr consumption',          'M',  'Output_Consumption'],
        labels_out.extend(['Domestic consumption', 'Industrial consumption', 'Livestock consumption'])
        parents_out.extend([ 'nonIrr consumption',  'nonIrr consumption',  'nonIrr consumption'])
        values_out.extend([np.sum(WB['Output_nonIrrConsumption'][0][1][2:]), np.sum(WB['Output_nonIrrConsumption'][1][1][2:]), np.sum(WB['Output_nonIrrConsumption'][2][1][2:])])
        colors_out.extend(['negative', 'negative', 'negative'])


        labels_out.extend(['Channel abstractions', 'Waterbody abstraction, conveyed'])
        labels_out.extend(['Channel (local) abstractions', 'Waterbody (local) abstraction, conveyed'])
        labels_out.extend(['Channel (non-local) abstractions', 'Waterbody (non-local) abstraction, conveyed'])
        
        parents_out.extend(['SW abstractions (applied)', 'SW abstractions (applied)'])
        parents_out.extend(['Channel abstractions', 'Waterbody abstraction, conveyed'])
        parents_out.extend(['Channel abstractions', 'Waterbody abstraction, conveyed'])
        
        values_out.extend([np.sum(WB['Input_SW_Consumption'][0][1][2:]), np.sum(WB['Input_SW_Consumption'][1][1][2:])])
        values_out.extend([np.sum(WB['Input_Channel_Consumption'][0][1][2:]), np.sum(WB['Input_Lake_Consumption'][0][1][2:])])
        values_out.extend([np.sum(WB['Input_Channel_Consumption'][1][1][2:]), np.sum(WB['Input_Lake_Consumption'][1][1][2:])])
        
        colors_out.extend(['positive', 'positive'])
        colors_out.extend(['positive', 'positive'])
        colors_out.extend(['positive', 'positive'])
        
    if section[0] == 'Withdrawal':

        labels_out.extend(['Domestic withdrawal', 'Industrial withdrawal', 'Livestock withdrawal'])
        parents_out.extend([ 'Withdrawal nonIrr',  'Withdrawal nonIrr',  'Withdrawal nonIrr'])
        values_out.extend([np.sum(WB['Input_nonIrrWithdrawal'][0][1][2:]), np.sum(WB['Input_nonIrrWithdrawal'][1][1][2:]), np.sum(WB['Input_nonIrrWithdrawal'][2][1][2:])])
        colors_out.extend(['positive', 'positive', 'positive'])
        

    labels_Circle_main =    ['Water flows total']+['Inputs',              'Outputs',            'Storage Change',     'Balance'] + labels_out 
    parents_Circle_main =   ['']+['Water flows total',   'Water flows total',  'Water flows total',  'Water flows total']           + parents_out
    values_Circle_main =    [total_input_adjusted+ total_output+magnitude_change+abs(balance)]+[total_input_adjusted,  total_output,         magnitude_change,    abs(balance)] + values_out 
    values_Parents_Circle_main = ["{:.1%}".format(values_Circle_main[i]/values_Parents_map_Circle[parents_Circle_main[i]]) for i in range(len(values_Circle_main))]
    #colors_Circle_main =    ['white']+['positive','negative'] +  ['positive' if total_change>0 else 'negative']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
    #colors_Circle_main =  ['white']+['positive','negative'] +  ['storage']+ ['positive' if balance>0 else 'negative'] + colors_out
    colors_Circle_main =  ['white']+['positive','negative'] +  ['storage']+ ['white'] + colors_out

    labels  =labels_Circle_main
    #print(labels)
    parents =parents_Circle_main
    #print(parents)
    values  =values_Circle_main
    #print(values)
    colors  =colors_Circle_main
    #print(colors)
    fractions = values_Parents_Circle_main
    #print(fractions)

    d={'labels':labels, 'parents':parents, 'values':values, 'colors':colors, 'fractions':fractions}
    df = pd.DataFrame(data=d)
    fig = px.sunburst(df, names='labels', parents='parents', values='values', color='colors', branchvalues='total', color_discrete_map=colors_Circle_main_discrete_map, hover_data=['fractions'])
    
    fig.update_layout(uniformtext=dict(minsize=15)) #, mode='hide'))
    fig.update_layout(height=700, title='Water Balance Circle,<br> whole<br><br> '+Title)
    fig.update_layout(
        title={'y':0.75, 'x':0.1, 'xanchor': 'left', 'yanchor': 'top'}) #, 'font_size':26})

    fig.update_layout(hoverlabel=dict(font_size=16), template='plotly_dark')
    

    #fig.show()        

    ####################       
    # Circle, simplified

    labels_Circle_simple = ['Inputs',              'Outputs',            'Storage Change',     'Balance'] + labels_simple
    parents_Circle_simple = ['Water flows total',   'Water flows total',  'Water flows total',  'Water flows total']           + parents_simple
    values_Circle_simple =  [total_input_adjusted,  total_output,         abs(total_change),    abs(balance)] + values_simple
    values_Parents_Circle_simple = ["{:.1%}".format(values_Circle_simple[i]/values_Parents_map_Circle[parents_Circle_main[i]]) for i in range(len(values_Circle_simple))]
    #colors_Circle_simple =  ['positive','negative'] +  ['storage']+ ['positive' if balance>0 else 'negative']+ colors_simple
    colors_Circle_simple =  ['positive','negative'] +  ['storage']+ ['white']+ colors_simple

    labels  =labels_Circle_simple
    parents =parents_Circle_simple
    values  =values_Circle_simple
    colors  =colors_Circle_simple
    fractions = values_Parents_Circle_simple
    d={'labels':labels, 'parents':parents, 'values':values, 'colors':colors, 'fractions':fractions}
    df = pd.DataFrame(data=d)
    fig_Circle_simple = px.sunburst(df, names='labels', parents='parents', values='values', color='colors', color_discrete_map=colors_Circle_main_discrete_map, hover_data=['fractions'])
    
    fig_Circle_simple.update_layout(height=700, title='Water Balance Circle,<br> glimpse<br><br> '+Title)
    #fig_Circle_simple.update_layout(uniformtext=dict(minsize=14, mode='hide'))
    fig_Circle_simple.update_layout(
        title={'y':0.75, 'x':0.05, 'xanchor': 'left', 'yanchor': 'top'}, template='plotly_dark') #, 'font_size':26})
    #fig_Circle_simple.show()

    ####################   
    # Circle, main_branchvalues NOT total

    labels_Circle_main =    ['Inputs',              'Outputs',            'Storage Change',     'Balance'] + labels_out 
    parents_Circle_main =   ['Water flows total',   'Water flows total',  'Water flows total',  'Water flows total']           + parents_out
    values_Circle_main =    [total_input_adjusted,  total_output,         magnitude_change,    abs(balance)] + values_out 
    values_Parents_Circle_main = ["{:.1%}".format(values_Circle_main[i]/values_Parents_map_Circle[parents_Circle_main[i]]) for i in range(len(values_Circle_main))]
    #colors_Circle_main =    ['white']+['positive','negative'] +  ['positive' if total_change>0 else 'negative']+ ['positive' if balance>0 else 'negative'] + ['positive'] + colors_out
    #colors_Circle_main =  ['white']+['positive','negative'] +  ['storage']+ ['positive' if balance>0 else 'negative'] + colors_out
    colors_Circle_main =  ['positive','negative'] +  ['storage']+ ['white'] + colors_out

    labels  =labels_Circle_main
    parents =parents_Circle_main
    values  =values_Circle_main
    colors  =colors_Circle_main
    fractions = values_Parents_Circle_main

    d={'labels':labels, 'parents':parents, 'values':values, 'colors':colors, 'fractions':fractions}

    df = pd.DataFrame(data=d)
    fig_Circle_whole_v2 = px.sunburst(df, names='labels', parents='parents', values='values', color='colors', color_discrete_map=colors_Circle_main_discrete_map, hover_data=['fractions'])
    # fig.update_layout(uniformtext=dict(minsize=15, mode='hide'))
    fig.update_layout(uniformtext=dict(minsize=15))
    fig_Circle_whole_v2.update_layout(height=700, title='Water Balance Circle,<br> whole v2<br><br> '+Title)
    fig_Circle_whole_v2.update_layout(
        title={'y':0.75, 'x':0.05, 'xanchor': 'left', 'yanchor': 'top'}) #, 'font_size':26})

    fig_Circle_whole_v2.update_layout(hoverlabel=dict(font_size=16))
    #fig_Circle_whole_v2.show()   

    ####################
    # Waterscape, Bar chart, daily water flows

    fig_Waterscape.update_traces(marker=dict(colorscale='twilight'))
    fig_Waterscape.update_layout(template='plotly_dark')
    #fig_Waterscape.show()

    # Bar chart, total, water balance

    Bar_WB=go.Figure(go.Bar(x=['Inputs', 'Outputs', 'Storage Change', 'Water balance'], y=[total_input_adjusted, total_output, abs(total_change), balance],marker=dict(color=['#636EFA', '#EF5538','#00CC96','black'])))
    Bar_WB.update_layout(title = 'Water flows, total<br><br> '+Title,
                      yaxis_title = 'Total Flow (M3/simulation) ', template='plotly_dark')

    #VARS = [[Input_current, 'Inputs', Total_input, 1, '', Input_daily], 
    #        [Output_current, 'Outputs', Total_output, -1, '', Output_daily], 
    #        [Storage_current, 'Storage', Total_change, -1, ' Change', Change_daily] ]
    # Line graph, and lines on Water Balance, waterscape
    Line_WB = go.Figure()
    Line_WB.update_layout(title = 'Water flows, daily<br><br> '+Title, xaxis_title = 'Days',yaxis_title = 'Flow (M3/day)', template='plotly_dark')

    for VAR in VARS:

        # Waterscape Line graph
        Line_WB.add_trace(go.Scatter(x=X, y=VAR[3]*VAR[5], fill='tozeroy', name=VAR[1]+VAR[4]))

        # Waterscape Bar chart, adding lines
        if VAR[1] == 'Inputs':
            color = '#636EFA'
        if VAR[1] == 'Outputs':
            color = '#EF553B'
        if VAR[1] == 'Storage':
            color = '#00CC96'

        fig_Waterscape.add_trace(go.Scatter(x=X, y=VAR[3]*VAR[5], name=VAR[1]+VAR[4], line=dict(color=color))) #,  #visible='legendonly')) {'positive':'rgb(128,177,211)','negative':'#EF5538', 'white':'white'}
    
    Line_WB.add_trace(go.Scatter(x=X, y=VARS[0][3]*VARS[0][5] + VARS[1][3]*VARS[1][5] + VARS[2][3]*VARS[2][5], fill='tozeroy', name= ' balance')) #VAR[1]+VAR[4]))
    ####################  
    # Table
    # intialise data of lists.
    data_WB = {'Water flows, total: '+Title:['Inputs', 'Outputs', 'Storage Change', 'Balance', 'Balance over all flows (%):'],'BMC':[  total_input_adjusted/1_000_000_000,   total_output/1_000_000_000,   total_change/1_000_000_000,   balance/1_000_000_000, percent_accurate_WB]}
    # Create DataFrame
    df = pd.DataFrame(data_WB)
    df=df.style.set_table_styles([{'selector': 'th', 'props': [('font-size', '15pt')]}]).set_properties(**{"font-size": "15pt"}).hide_index()



    ####################   
    # Print examples.
    #fig_Circle_simple.show()
    #fig.show()
    #fig_Circle_whole_v2.show() 
    #Line_WB.show()
    #fig_Waterscape.show()
    #Bar_WB.show()
    #display(df)
    #po.iplot(figure)

    Figures['fig_Circle_simple_'+section[0]]= fig_Circle_simple
    Figures['fig_'+section[0]]= fig
    Figures['fig_Circle_whole_v2_'+section[0]]= fig_Circle_whole_v2
    Figures['Line_WB_'+section[0]]= Line_WB
    
    Figures['fig_Waterscape_'+section[0]]= fig_Waterscape
    Figures['Bar_WB_'+section[0]]= Bar_WB
    Figures['df_'+section[0]]= df
    Figures['figure_'+section[0]]= figure
    
    #'Lakes and Reservoirs'
    

In [ ]:
#Line_WB.show()

# Water balance

A water balance is used to understand how water flows through a system. This includes how much rainfall goes into the soil and evapotranspires and how much flows as river discharge. What is groundwater recharge and what is groundwater dischrage as baseflow and capillary rise? The water balance is an application of the conservation of mass, and thus the inputs to the system should equal the outputs from the system plus whatever stayed in the system. 

$\text{Inputs} - \left(\text{Outputs} +\Delta\text{Storage}\right) = 0$


## Water balance circle

The water balance is illustrated below in a sunburst pie chart. 
1. If the water balance circle is closed (no blank white gap), then the water balance is also closed. 
2. The dominant inputs and outputs of the system are labelled, while the other variables and their values are seen by hovering the cursor over the smaller slices.
3. Inputs are blue, outputs are red, and storage is green. The partitioned storage change colours relate to the storage either increasing (input = blue) or decreasing (output = red).

In [ ]:
#Figures['fig_Circle_whole_v2_'].show()

#Figures['fig_Circle_simple_'].show()
#Figures['fig_'].show()


## Water balance: Bar chart and Table

The total water balance visualised with the water balance circle presented above, is presented here as a bar chart and table.

The Balance over all flows (%) entry in the table is the following calculation: $\frac{\text{Inputs} - \left(\text{Outputs} +\Delta\text{Storage}\right)}{\frac{1}{2} \left(\text{Inputs} + \text{Outputs} +\Delta\text{Storage}\right)}*100$



In [ ]:
#Figures['Bar_WB_'].show()
#display(Figures['df_'])

## Water balance waterscape: daily fluxes

In this Water balance waterscape, the inputs are positive, while the outputs and increases in storage are negative. If the water balance is closed, the area of the top half of the graph should equal that of the bottom half.

The different flows can be visulised in different combinations: 
- Deselect flows to visualise by clicking on their legend entry.
- A single flow can be isolated by double clicking on the legend entry, and double-clicking a single entry includes all the flows in the visulaistion.
- Graphs showing the total inputs, outputs, and changes in storage daily can be visualised by activating the the Inputs, Outputs, and Change in Storage entries in the legend. These are initially turned off.

## Specific flows and storages

The specific flow and storage variables used in CWatM related to this water balance can be visualised over the simulation period. 

In [ ]:
#po.init_notebook_mode() 

#po.iplot(Figures['figure_'])

Water to satisfy requests are potentially available from five main sources: 
1.	local rivers and waterbodies, 
2.	shared river water within “lift areas”, 
3.	reservoir water within “command areas”, and
4.	local groundwater pumping


The amount asked of each water source to satisfy requests is determined cell-specific using six "abstraction fractions". Abstraction fractions distribute the share of surface and groundwaters to satisfy water requests.
After the specific surface water abstractions take place the total withdrawn surface water is used to satisfy first urban & rural requests, and then those of irrigation. The full remaining urban & rural water request will be asked of groundwater. Up to all of the remaining irrigation request will be asked of groundwater. 

Abstraction Fraction | Description | Variable
--- | --- | ---
$\text{frac}_{Irr}^{sw}$ | fraction of irrigation request from surface water | swAbstractionFraction_Irr 
$\text{frac}_{nonIrr}^{sw}$ | fraction of domestic request from surface water|  swAbstractionFraction_nonIrr
$\text{frac}_{local}^{sw}$| fraction of surface water request from local water | swAbstractionFraction_Local
$\text{frac}_{lift}^{sw}$ |fraction of surface water request from lift area | swAbstractionFraction_Lift 
$\text{frac}_{command}^{sw}$ | fraction of surface water request from command area | swAbstractionFraction_nonLocal
$\text{frac}_{Irr}^{gw}$| fraction of irrigation request from groundwater | gwAbstractionFraction_Irr 